In [6]:
import pandas as pd
import numpy as np 

In [7]:
## read and format global datasets 

## Global dataset for average CO2, units: 
global_co2 = pd.read_csv('nasa/co2_formatted.csv')
global_co2['Year'] = [int(i) for i in global_co2.date]
global_co2 = global_co2.groupby('Year').mean()
global_co2.reset_index(inplace=True)
global_co2 = global_co2.drop(['#', 'Unnamed: 1', 'date', 'alized', 'Unnamed: 5', 'of days', 'mon mean'], axis =1)
global_co2.columns = ['Year', 'co2_avg']
global_co2 = global_co2[(global_co2.Year >= 1965) & (global_co2.Year <= 2019)]
global_co2.reset_index(inplace=True)
global_co2 = global_co2.drop(['index'], axis=1)

## Global dataset for temperature 
global_temp = pd.read_csv('nasa/global_temp_formatted.csv')
global_temp = global_temp.drop(['No_Smoothing'], axis=1)
global_temp.columns = ['Year', 'temp']
global_temp = global_temp[(global_temp.Year >= 1965) & (global_temp.Year <= 2019)]#benchmarked by CO2 data
global_temp.reset_index(inplace = True)
global_temp = global_temp.drop(['index'], axis=1)

## Global population dataset 
global_pop = pd.read_csv('absolute-increase-global-population.csv')
global_pop = global_pop.drop(['Entity', 'Code',
                              'Projected absolute population increase (OWID based on HYDE & UN)'], axis =1)
global_pop.dropna(inplace= True)
global_pop.columns = ['Year', 'pop_increase']
global_pop = global_pop[(global_pop.Year >= 1965) & (global_pop.Year <= 2019)] #benchmarked by CO2 data
global_pop.reset_index(inplace = True)
global_pop['pop'] = ''
pop_1957 = 2873306090
global_pop.loc[0, 'pop'] = pop_1957 + global_pop.loc[0, 'pop_increase']

for i in range(1,len(global_pop)):
    global_pop.loc[i, 'pop'] = global_pop.loc[i-1, 'pop'] + global_pop.loc[i, 'pop_increase']

# manually append missing years 
missing_yrs = [2017, 2018, 2019]
missing_pop = [7547858925.0, 7631091040.0, 7713468100.0] 
missing_df = pd.DataFrame({'Year': missing_yrs, 'pop_increase': np.zeros([3,]), 'pop': missing_pop })
global_pop = global_pop.append(missing_df)
global_pop.reset_index(inplace=True)
global_pop = global_pop.drop(['level_0', 'index', 'pop_increase'], axis=1)

## Global Energy Dataset 
# Data is consistent (annual) from 1965 onwards 
# source: https://ourworldindata.org/energy-mix
## info: Primary energy is calculated based on the 'substitution method' which takes account of the inefficiencies 
### in fossil fuel production by converting non-fossil energy into the energy inputs required if they had the same 
### conversion losses as fossil fuels.

global_energy = pd.read_csv('global-energy-substitution.csv')
global_energy = global_energy[global_energy.Year >= 1965]

global_energy['fossil_fuels (TWh)'] = global_energy['Oil (TWh; substituted energy)'] 
+ global_energy['Coal (TWh; substituted energy)']
+ global_energy['Gas (TWh; substituted energy)']

global_energy['renewables (TWh)'] = global_energy['Wind (TWh; substituted energy)'] 
+ global_energy['Hydropower (TWh; substituted energy)'] 
+ global_energy['Nuclear (TWh; substituted energy)'] 
+ global_energy['Traditional bimass (TWh; substituted energy)']
+ global_energy['Other renewables (TWh; substituted energy)'] 
+ global_energy['Biofuels (TWh; substituted energy)'] 
+ global_energy['Solar (TWh; substituted energy)']

global_energy = global_energy.drop(['Entity', 'Code', 'Wind (TWh; substituted energy)',
       'Oil (TWh; substituted energy)', 'Nuclear (TWh; substituted energy)',
       'Hydropower (TWh; substituted energy)',
       'Traditional bimass (TWh; substituted energy)',
       'Other renewables (TWh; substituted energy)',
       'Biofuels (TWh; substituted energy)', 'Solar (TWh; substituted energy)',
       'Coal (TWh; substituted energy)', 'Gas (TWh; substituted energy)'], axis=1)

global_energy['ALL Fuels'] = global_energy['fossil_fuels (TWh)'] + global_energy['renewables (TWh)']
global_energy.reset_index(inplace=True)
global_energy = global_energy.drop(['index'], axis =1)

In [8]:
print(global_co2.columns)
print(global_temp.columns)
print(global_pop.columns)
print(global_energy.columns)

Index(['Year', 'co2_avg'], dtype='object')
Index(['Year', 'temp'], dtype='object')
Index(['Year', 'pop'], dtype='object')
Index(['Year', 'fossil_fuels (TWh)', 'renewables (TWh)', 'ALL Fuels'], dtype='object')


In [9]:
## Aggregate Global Dataset, 1965-2019

df_Global = pd.DataFrame({'Year': global_energy.Year, 'pop': global_pop['pop'], 
                          'co2_avg': global_co2['co2_avg'], 'temp': global_temp['temp'], 
                          'fossil_fuels (TWh)': global_energy['fossil_fuels (TWh)'], 
                          'renewables (Twh)': global_energy['renewables (TWh)'], 
                          'All Fuels': global_energy['ALL Fuels']})

## produce csv of data : 

df_Global.to_csv('output/global_data.csv')

### Country-Level data prep


In [74]:
country_GHG = pd.read_csv('OECD/AIR_GHG.csv')
country_pop = pd.read_csv('OECD/POP_1965-2019.csv')
country_energy = pd.read_csv('OECD/primary-energy.csv')

In [70]:
print(country_GHG.columns)
print(country_pop.columns)
print(country_energy.columns)
print(country_co2.columns)

Index(['COU', 'Country', 'POL', 'Pollutant', 'VAR', 'Variable', 'YEA', 'Year',
       'Unit Code', 'Unit', 'PowerCode Code', 'PowerCode',
       'Reference Period Code', 'Reference Period', 'Value', 'Flag Codes',
       'Flags'],
      dtype='object')
Index(['LOCATION', 'Country', 'SEX', 'Sex', 'AGE', 'Age', 'VAR', 'Variant',
       'TIME', 'Time', 'Unit Code', 'Unit', 'PowerCode Code', 'PowerCode',
       'Reference Period Code', 'Reference Period', 'Value', 'Flag Codes',
       'Flags'],
      dtype='object')
Index(['Entity', 'Code', 'Year', 'Primary energy consumption (TWh)'], dtype='object')
Index(['iso_code', 'country', 'year', 'co2', 'co2_growth_prct',
       'co2_growth_abs', 'consumption_co2', 'trade_co2', 'trade_co2_share',
       'co2_per_capita', 'consumption_co2_per_capita', 'share_global_co2',
       'cumulative_co2', 'share_global_cumulative_co2', 'co2_per_gdp',
       'consumption_co2_per_gdp', 'co2_per_unit_energy', 'cement_co2',
       'coal_co2', 'flaring_co2', 'gas_c

In [123]:
country_all = pd.read_csv('OECD/CO2_emissions.csv')
country_all = country_all[(country_all.year >= 1965) & (country_all.year < 2017)]
## source: https://github.com/owid/co2-data {Our world in Data}

## columns that we want to keep: 
cols = ['country', 'year', 'co2', 'co2_per_gdp', 'share_global_co2', 
        'co2_per_capita', 'co2_per_unit_energy', 'primary_energy_consumption', 
        'energy_per_capita', 'energy_per_gdp', 'population', 'gdp']

country_all = country_all[cols]

## countries to keep 

country = ['Australia', 'Austria', 'Belgium', 'Brazil', 'Bulgaria', 'Canada',
       'Chile', "China (People's Republic of)", 'Colombia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Finland', 'France',
       'Germany', 'Greece', 'Hungary', 'Iceland', 'India', 'Indonesia',
       'Ireland', 'Italy', 'Japan', 'Korea', 
       'Luxembourg',  'Mexico', 'Netherlands',
       'New Zealand', 'Norway', 'OECD - Total', 'Poland', 'Portugal',
       'Romania', 'Saudi Arabia', 'Slovak Republic', 'South Africa',
       'Spain', 'Sweden', 'Switzerland', 'Turkey', 'United Kingdom',
       'United States']

check = []
for i in country:
    df = country_all[country_all.country == i]
    check.append(df)

country_all = pd.concat(check)
country_all.reset_index(inplace = True)

#export to csv 
country_all.to_csv('output/country-level.csv')

In [136]:
print(country_GHG.columns)
print(country_pop.columns)
country_GHG.head()

Index(['COU', 'Country', 'POL', 'Pollutant', 'VAR', 'Variable', 'YEA', 'Year',
       'Unit Code', 'Unit', 'PowerCode Code', 'PowerCode',
       'Reference Period Code', 'Reference Period', 'Value', 'Flag Codes',
       'Flags'],
      dtype='object')
Index(['LOCATION', 'Country', 'SEX', 'Sex', 'AGE', 'Age', 'VAR', 'Variant',
       'TIME', 'Time', 'Unit Code', 'Unit', 'PowerCode Code', 'PowerCode',
       'Reference Period Code', 'Reference Period', 'Value', 'Flag Codes',
       'Flags'],
      dtype='object')


,COU,Country,POL,Pollutant,VAR,Variable,YEA,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,GHG,Greenhouse gases,TOTAL,Total emissions excluding LULUCF,1990,1990,T_CO2_EQVT,Tonnes of CO2 equivalent,3,Thousands,NaN,NaN,424998.381,NaN,NaN
1,AUS,Australia,GHG,Greenhouse gases,TOTAL,Total emissions excluding LULUCF,1991,1991,T_CO2_EQVT,Tonnes of CO2 equivalent,3,Thousands,NaN,NaN,426015.210,NaN,NaN
2,AUS,Australia,GHG,Greenhouse gases,TOTAL,Total emissions excluding LULUCF,1992,1992,T_CO2_EQVT,Tonnes of CO2 equivalent,3,Thousands,NaN,NaN,430216.380,NaN,NaN
3,AUS,Australia,GHG,Greenhouse gases,TOTAL,Total emissions excluding LULUCF,1993,1993,T_CO2_EQVT,Tonnes of CO2 equivalent,3,Thousands,NaN,NaN,430612.556,NaN,NaN
4,AUS,Australia,GHG,Greenhouse gases,TOTAL,Total emissions excluding LULUCF,1994,1994,T_CO2_EQVT,Tonnes of CO2 equivalent,3,Thousands,NaN,NaN,430653.332,NaN,NaN


In [135]:
country_GHG = pd.read_csv('OECD/AIR_GHG.csv')

cols = ['Country', 'Year', 'Value']
country_ghg = country_GHG[cols]
country_ghg.columns = ['Country', 'Year', 'GHG']


country = ['Australia', 'Austria', 'Belgium', 'Brazil', 'Bulgaria', 'Canada',
       'Chile', "China (People's Republic of)", 'Colombia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Finland', 'France',
       'Germany', 'Greece', 'Hungary', 'Iceland', 'India', 'Indonesia',
       'Ireland', 'Italy', 'Japan', 'Korea', 
       'Luxembourg',  'Mexico', 'Netherlands',
       'New Zealand', 'Norway', 'OECD - Total', 'Poland', 'Portugal',
       'Romania', 'Saudi Arabia', 'Slovak Republic', 'South Africa',
       'Spain', 'Sweden', 'Switzerland', 'Turkey', 'United Kingdom',
       'United States']

check = []
for i in country:
    df = country_ghg[country_ghg.Country == i]
    check.append(df)
    
country_ghg = pd.concat(check)
country_ghg.reset_index(inplace = True)

country_ghg.to_csv('output/country-all_ghg.csv')